In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
file = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
X,y = file[file.columns[:-1]], file[file.columns[-1:]]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
scaler = MaxAbsScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
def base_model():
    inputs = Input(shape=(13,), name='input')
    bn1 = BatchNormalization()(inputs)
    dense1 = Dense(256, kernel_regularizer = regularizers.l1(1e-4), activation='relu')(bn1)
    dropout1 = Dropout(rate=0.2)(dense1)
    
    bn2 = BatchNormalization()(dropout1)
    dense2 = Dense(128,kernel_regularizer = regularizers.l1(1e-5), activation='relu')(bn2)
    dropout2 = Dropout(rate=0.2)(dense2)
    
    dense3 = Dense(128,kernel_regularizer = regularizers.l1(1e-5), activation='relu')(dropout2)
    outputs = Dense(1, activation='sigmoid', name='output')(dense3)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model
def visualize(metric):
    plt.plot(history.history[metric], label=metric)
    plt.plot(history.history['val_'+metric], label='val_'+metric)
    plt.legend()

In [ ]:
model = base_model()
# tf.keras.utils.plot_model(model)
metrics = [ tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),tf.keras.metrics.Recall(name = 'recall'),tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.TruePositives(name = 'tp'), tf.keras.metrics.TrueNegatives(name = 'tn'),
           tf.keras.metrics.FalsePositives(name = 'fp'), tf.keras.metrics.FalseNegatives(name = 'fn')]
model.compile(optimizer=RMSprop(learning_rate=0.01, momentum=0.01), loss='binary_crossentropy', metrics=metrics)
history = model.fit(x_train, y_train, epochs=200, batch_size=64, verbose=0, validation_data=(x_test,y_test))
result = model.evaluate(x_test,y_test, verbose=0, return_dict = True)

In [ ]:
visualize('accuracy')

In [ ]:
result